In [1]:
import logging
import random

import numpy as np

import gym
from gym.envs.board_game import go

logging.basicConfig(level=logging.INFO)

# ACS2 in Go environment
## Playing Go with Pachi
OpenAI engine for playing Go uses *Pachi framework* (can be also used for other board games such as Weiqi or Baduk).

From the [docs](http://repo.or.cz/pachi.git/blob_plain/HEAD:/README) we can read that: 

> The default engine plays by Chinese rules and should be about 7d KGS strength on 9x9. On 19x19 (using e.g. six-way Intel i7), it can hold a solid KGS 2d rank.  When using a large cluster (64 machines, 20 cores each), it maintains KGS 3d to 4d and has won e.g. a 7-stone handicap game against Zhou Junxun 9p. 

> By default, Pachi currently uses the UCT engine that combines Monte Carlo approach with tree search; UCB1AMAF tree policy using the RAVE method is used for tree search, while the Moggy playout policy using 3x3 patterns and various tactical checks is used for the semi-random Monte Carlo playouts.  Large-scale board patterns are used in the tree search.

> At the same time, we keep trying a wide variety of other approaches and enhancements. Pachi is an active research platform and quite a few improvements have been already achieved. We rigorously play-test new features and enable them by default only when they give a universal strength boost.

Begin with initializing the environment into initial state and visualizing state into console.

In [2]:
env = gym.make('Go9x9-v0')

# Reset the state
state = env.reset()

# Render current state
print(env._state)

INFO:gym.envs.registration:Making new env: Go9x9-v0


To play: black
Move:   0  Komi: 0.0  Handicap: 0  Captures B: 0 W: 0
      A B C D E F G H J  
    +-------------------+
  9 | . . . . . . . . . |
  8 | . . . . . . . . . |
  7 | . . . . . . . . . |
  6 | . . . . . . . . . |
  5 | . . . . . . . . . |
  4 | . . . . . . . . . |
  3 | . . . . . . . . . |
  2 | . . . . . . . . . |
  1 | . . . . . . . . . |
    +-------------------+


Pachi uses it's own way of describing state of the Go play. To represent action we need to convert it from symbolic symbol (such as *A8*) to internal numeric representation. OpenAi delivers function for mapping but in order to know what actions are possible we need to know mapping between all representations.

In [3]:
def moves_9x9():
    """
    Return a list of all available moves (such as A8) on 9x9 board.
    """
    rows = [chr(i) for i in range(ord('A'),ord('H')+1)]
    cols = list(range(1, 8 + 1))
    actions = []

    for row in rows:
        for col in cols:
            actions.append("{}{}".format(row, col))

    return actions

We can define a function performing action mapping from numbers from `0-64` to Pachi internal representation

In [4]:
def map_action(action_idx):
    """
    Maps action with given index (0, num_actions) to Pachi representation.
    """
    moves = [(move, go.str_to_action(env._state.board, move)) for move in moves_9x9()]
    return moves[action_idx][1]

To perform a move let's pick up a random opening action

In [5]:
action_idx = random.randint(0, 63)
action = map_action(action_idx)

print("Action idx: [{}] - Pachi action: [{}]".format(action_idx, action))

Action idx: [29] - Pachi action: [30]


Now we can execute it getting back all possible information

In [6]:
state, reward, done, info = env.step(action)

print("Reward: [{}]".format(reward))
print("Done: [{}]".format(done))
print(info['state'])

Reward: [0.0]
Done: [False]
To play: black
Move:   2  Komi: 0.0  Handicap: 0  Captures B: 0 W: 0
      A B C D E F G H J  
    +-------------------+
  9 | . . . . . . . . . |
  8 | . . . . . . . . . |
  7 | . . . . O). . . . |
  6 | . . . X . . . . . |
  5 | . . . . . . . . . |
  4 | . . . . . . . . . |
  3 | . . . . . . . . . |
  2 | . . . . . . . . . |
  1 | . . . . . . . . . |
    +-------------------+


## ACS2

In [7]:
# Import PyALCS code from local path
import sys
sys.path.append('/Users/khozzy/Projects/pyalcs')

from alcs import ACS2, ACS2Configuration

# Enable automatic module reload
%load_ext autoreload
%autoreload 2

Configure agent. In this case we have to specify the classifier length (state of the board), number of possible actions (all possible moves). The state returned by the environment is not suitable to be used in ACS2 classifier. Therefore a special mapper function is created.

In [8]:
def map_state(state):
    """
    Returns a flatten array of board state of given state.
    Black stones are represented as 'B', whites as 'W'
    """
    black_stones = state[0]
    white_stones = state[1] * 2
    
    board = (black_stones + white_stones).astype('str')
    board[board == '1'] = 'B'
    board[board == '2'] = 'W'
    
    return list(board.flatten())

We want our agent to focus more on exploration than exploitation (`epsilon` parameter).

In [15]:
CLASSIFIER_LENGTH=env._state.board.size ** 2
NUMBER_OF_POSSIBLE_ACTIONS=len(moves_9x9())

cfg = ACS2Configuration(
    classifier_length=CLASSIFIER_LENGTH,
    number_of_possible_actions=NUMBER_OF_POSSIBLE_ACTIONS,
    perception_mapper_fcn=map_state,
    action_mapper_fcn=map_action,
    epsilon=0.7
)

print(cfg)

ACS2Configuration:
	- Classifier length: [81]
	- Number of possible actions: [64]
	- Classifier wildcard: [#]
	- Perception mapper function: [<function map_state at 0x10c12e620>]
	- Action mapper function: [<function map_action at 0x10d215620>]
	- Do GA: [False]
	- Do subsumption: [True]
	- Beta: [0.05]
	- ...
	- Epsilon: [0.7]


Now we can build the agent using previously defined configuration.

In [16]:
agent = ACS2(cfg)

Explore best action.

In [17]:
population, metrics = agent.explore(env, 10)

In [18]:
population

[#################################0#########################################0#####-24-#################################W#########################################B##### @ 0x10dcbfdd8,
 ####################0#################0##########################################-20-####################W#################B########################################## @ 0x10dcfb048,
 ##############################################0####0#############################-11-##############################################B####W############################# @ 0x10dcbf550,
 #############0#################0#################################################-37-#############W#################B################################################# @ 0x10dcbf198,
 ############0###########################################################0########-0-############W###########################################################B######## @ 0x10dcdcc88,
 ####################################################################0#####0######-16-